In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Legend, LogColorMapper, GeoJSONDataSource, LinearColorMapper, ColorBar, Range1d, NumeralTickFormatter, HoverTool, LabelSet, Panel, Tabs, Slider, CustomJS
from bokeh.models.widgets import TableColumn, DataTable, NumberFormatter, Dropdown, Select, RadioButtonGroup
from bokeh.palettes import Category20c, Spectral, GnBu, brewer, PRGn, RdYlGn
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Viridis6 as palette
from bokeh.transform import cumsum
from bokeh.resources import INLINE
import bokeh.io

import geopandas as gpd
import pycountry
import pycountry_convert as pc
import json

from powerbiclient import QuickVisualize, get_dataset_config, Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

from rich.console import Console

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv(r'data/REGEN_%_RAW.csv', encoding_errors='replace', header=2)

In [8]:
ren_df = pd.read_csv(r'data/REGEN_GWh_RAW.csv', encoding_errors='replace', header=1)

In [9]:
df

,Region/country/area,Indicator,2018,2019,2020,2021,2022
0,World,RE share of electricity capacity (%),33.16,34.60,36.66,38.42,40.26
1,World,RE share of electricity generation (%),24.87,25.85,27.65,27.79,0.00
2,Africa,RE share of electricity capacity (%),21.61,21.71,22.51,22.94,23.61
3,Africa,RE share of electricity generation (%),19.86,20.66,22.31,22.74,0.00
4,Algeria,RE share of electricity capacity (%),3.09,2.80,2.63,2.28,2.26
...,...,...,...,...,...,...,...
463,Suriname,RE share of electricity generation (%),56.19,51.35,39.49,53.68,0.00
464,Uruguay,RE share of electricity capacity (%),75.80,76.00,76.02,76.06,76.12
465,Uruguay,RE share of electricity generation (%),97.32,98.04,93.92,84.53,0.00
466,Venezuela (Bolivarian Republic of),RE share of electricity capacity (%),48.82,49.14,48.68,48.68,48.68


In [10]:
gen_df = df.query('Indicator=="RE share of electricity generation (%)"')

In [11]:
gen_df

,Region/country/area,Indicator,2018,2019,2020,2021,2022
1,World,RE share of electricity generation (%),24.87,25.85,27.65,27.79,0.0
3,Africa,RE share of electricity generation (%),19.86,20.66,22.31,22.74,0.0
5,Algeria,RE share of electricity generation (%),1.02,1.03,1.06,0.91,0.0
7,Angola,RE share of electricity generation (%),69.76,71.87,73.70,73.70,0.0
9,Benin,RE share of electricity generation (%),1.64,0.40,0.40,0.22,0.0
...,...,...,...,...,...,...,...
459,Peru,RE share of electricity generation (%),60.89,60.51,64.38,61.67,0.0
461,South Georgia and S Sandwich Islands,RE share of electricity generation (%),98.60,98.96,98.96,100.00,0.0
463,Suriname,RE share of electricity generation (%),56.19,51.35,39.49,53.68,0.0
465,Uruguay,RE share of electricity generation (%),97.32,98.04,93.92,84.53,0.0


In [12]:
ren_df['unit'] = 'gwh'

ren_df.columns = [
    'country',
    'technology',
    '2018',
    '2019',
    '2020',
    '2021',
    'unit'
]
ren_df['country'] = ren_df['country'].copy().apply(lambda x: x.lower())
ren_df['technology'] = ren_df['technology'].copy().apply(lambda x: x.lower())

ren_df

,country,technology,2018,2019,2020,2021,unit
0,world,total renewable energy,6627718.734,6994720.276,7455905.333,7857802.526,gwh
1,world,hydropower,4306776.270,4352825.868,4477721.630,4400679.013,gwh
2,world,renewable hydropower,4189346.361,4238697.906,4357351.497,4275374.687,gwh
3,world,pumped storage,117429.909,114127.962,120370.133,125304.326,gwh
4,world,marine,1021.241,995.175,983.975,970.024,gwh
...,...,...,...,...,...,...,...
4726,venezuela (bolivarian republic of),bagasse,0.000,0.000,0.000,0.000,gwh
4727,venezuela (bolivarian republic of),other solid biofuels,0.000,0.000,0.000,0.000,gwh
4728,venezuela (bolivarian republic of),liquid biofuels,0.000,0.000,0.000,0.000,gwh
4729,venezuela (bolivarian republic of),biogas,0.000,0.000,0.000,0.000,gwh


In [13]:
gen_df.drop(['Indicator', '2022'], axis=1, inplace=True)
gen_df['technology'] = 'total_perc'
gen_df['unit'] = 'perc'

gen_df.columns = [
    'country',
    '2018',
    '2019',
    '2020',
    '2021',
    'technology',
    'unit'
]

gen_df['country'] = gen_df['country'].copy().apply(lambda x: x.lower())
gen_df['technology'] = gen_df['technology'].copy().apply(lambda x: x.lower())

gen_df

,country,2018,2019,2020,2021,technology,unit
1,world,24.87,25.85,27.65,27.79,total_perc,perc
3,africa,19.86,20.66,22.31,22.74,total_perc,perc
5,algeria,1.02,1.03,1.06,0.91,total_perc,perc
7,angola,69.76,71.87,73.70,73.70,total_perc,perc
9,benin,1.64,0.40,0.40,0.22,total_perc,perc
...,...,...,...,...,...,...,...
459,peru,60.89,60.51,64.38,61.67,total_perc,perc
461,south georgia and s sandwich islands,98.60,98.96,98.96,100.00,total_perc,perc
463,suriname,56.19,51.35,39.49,53.68,total_perc,perc
465,uruguay,97.32,98.04,93.92,84.53,total_perc,perc


In [14]:
country_out = {
    't�rkiye': 'turkiye',
    'r�union': 'reunion',
    'iran (islamic republic of)': 'iran',
    "cote d'ivoire": 'ivory coast',
    'bonaire, sint eustatius and saba': 'caribbean netherlands',
    'viet nam': 'vietnam',
    'south georgia and s sandwich islands': 'south georgia island',
    'venezuela (bolivarian republic of)': 'venezuela'
}

In [15]:
gen_df.country.unique().tolist()

['world',
 'africa',
 'algeria',
 'angola',
 'benin',
 'botswana',
 'burkina faso',
 'burundi',
 'cabo verde',
 'cameroon',
 'central african republic',
 'chad',
 'comoros',
 'democratic republic of the congo',
 'congo',
 "cote d'ivoire",
 'djibouti',
 'egypt',
 'equatorial guinea',
 'eritrea',
 'eswatini',
 'ethiopia',
 'gabon',
 'gambia',
 'ghana',
 'guinea',
 'guinea-bissau',
 'kenya',
 'lesotho',
 'liberia',
 'libya',
 'madagascar',
 'malawi',
 'mali',
 'mauritania',
 'mauritius',
 'mayotte',
 'morocco',
 'mozambique',
 'namibia',
 'niger',
 'nigeria',
 'reunion',
 'rwanda',
 'sao tome and principe',
 'senegal',
 'seychelles',
 'sierra leone',
 'somalia',
 'south africa',
 'south sudan',
 'sudan',
 'united republic of tanzania',
 'togo',
 'tunisia',
 'uganda',
 'zambia',
 'zimbabwe',
 'asia',
 'afghanistan',
 'bangladesh',
 'bhutan',
 'brunei darussalam',
 'cambodia',
 'china',
 'china, hong kong sar',
 'chinese taipei',
 'india',
 'indonesia',
 'japan',
 'kazakhstan',
 "democratic

In [16]:
en_df = pd.concat([ren_df, gen_df])

In [17]:
np.max([3,52,3]+[3,2,6])

52

### Percentage sustainable energy generated in the world

In [18]:
round(en_df.query('country=="world" and technology==["total_perc", "total renewable energy"]'),2)

,country,technology,2018,2019,2020,2021,unit
0,world,total renewable energy,6627718.73,6994720.28,7455905.33,7857802.53,gwh
1,world,total_perc,24.87,25.85,27.65,27.79,perc


### Percentage sustainable energy generated in the world

In [19]:
def country_perc(data, name):
    """Percentage energy data from each country"""
    data = data.query('country==@name and technology!="total_perc"')
    
    dict_co = en_df.query('country==@name and technology!="total_perc" and technology=="total renewable energy"')
    
    data['2018'] = data['2018'].apply(lambda s: round(s*100/dict_co['2018'], 2)).copy()
    data['2019'] = data['2019'].apply(lambda s: round(s*100/dict_co['2019'], 2)).copy()
    data['2020'] = data['2020'].apply(lambda s: round(s*100/dict_co['2020'], 2)).copy()
    data['2021'] = data['2021'].apply(lambda s: round(s*100/dict_co['2021'], 2)).copy()
    
    return data[['technology', '2018', '2019', '2020', '2021']]

def country_data(data, name):
    """Percentage energy data from each country"""
    data = data.query('country==@name and technology!="total_perc"')
    
    dict_co = en_df.query('country==@name and technology!="total_perc" and technology=="total renewable energy"')
    
    return data[['technology', '2018', '2019', '2020', '2021']]

### World

In [20]:
country_data(en_df,"world")

,technology,2018,2019,2020,2021
0,total renewable energy,6627718.734,6994720.276,7455905.333,7857802.526
1,hydropower,4306776.270,4352825.868,4477721.630,4400679.013
2,renewable hydropower,4189346.361,4238697.906,4357351.497,4275374.687
3,pumped storage,117429.909,114127.962,120370.133,125304.326
4,marine,1021.241,995.175,983.975,970.024
5,wind,1258795.590,1412376.716,1588477.108,1838263.605
6,onshore wind energy,1190508.725,1327739.230,1484406.626,1700649.956
7,offshore wind energy,68286.865,84637.486,104070.482,137613.649
8,solar,562505.340,693552.047,842411.926,1033925.911
9,solar photovoltaic,550458.288,679386.506,829252.966,1020297.378


In [21]:
country_perc(en_df,"world")

,technology,2018,2019,2020,2021
0,total renewable energy,100.00,100.00,100.00,100.00
1,hydropower,64.98,62.23,60.06,56.00
2,renewable hydropower,63.21,60.60,58.44,54.41
3,pumped storage,1.77,1.63,1.61,1.59
4,marine,0.02,0.01,0.01,0.01
5,wind,18.99,20.19,21.30,23.39
6,onshore wind energy,17.96,18.98,19.91,21.64
7,offshore wind energy,1.03,1.21,1.40,1.75
8,solar,8.49,9.92,11.30,13.16
9,solar photovoltaic,8.31,9.71,11.12,12.98


### European Union

In [22]:
country_data(en_df, "european union")

,technology,2018,2019,2020,2021
3458,total renewable energy,942516.837,980447.660,1062234.635,1077966.153
3459,hydropower,371416.598,346517.913,376372.482,375804.480
3460,renewable hydropower,345063.592,321134.729,348066.085,349273.315
3461,pumped storage,26353.006,25383.184,28306.397,26531.165
3462,marine,500.230,499.164,508.964,502.991
3463,wind,320686.963,367238.855,397926.695,387026.095
3464,onshore wind energy,288668.750,326961.662,350479.119,339222.575
3465,offshore wind energy,32018.213,40277.193,47447.576,47803.520
3466,solar,113581.268,124423.125,145662.317,164353.426
3467,solar photovoltaic,108701.793,118727.650,140657.842,159164.951


### Asia

In [23]:
country_data(en_df, "asia")

,technology,2018,2019,2020,2021
1121,total renewable energy,2648726.331,2877090.743,3095750.575,3418454.153
1122,hydropower,1775331.300,1847348.374,1923623.034,1917753.758
1123,renewable hydropower,1721622.100,1794974.174,1868803.834,1855705.558
1124,pumped storage,53709.200,52374.200,54819.200,62048.200
1125,marine,491.506,480.506,463.506,461.506
1126,wind,440338.328,491470.863,554832.120,747547.721
1127,onshore wind energy,430702.543,479193.387,539040.019,693396.847
1128,offshore wind energy,9635.785,12277.476,15792.101,54150.874
1129,solar,292376.516,369930.700,441714.531,549775.735
1130,solar photovoltaic,292007.234,369555.941,441334.294,549394.622


### Africa

In [24]:
country_data(en_df, "africa")

,technology,2018,2019,2020,2021
19,total renewable energy,162233.619,173858.339,183625.307,194361.046
20,hydropower,140549.073,147724.740,152606.568,159208.662
21,renewable hydropower,135682.432,142391.240,147322.868,154058.317
22,pumped storage,4866.641,5333.500,5283.700,5150.345
23,marine,0.000,0.000,0.000,0.000
24,wind,7845.576,9704.169,10563.129,12337.638
25,onshore wind energy,7845.576,9704.169,10563.129,12337.638
26,offshore wind energy,0.000,0.000,0.000,0.000
27,solar,10215.359,12992.740,17238.191,19620.289
28,solar photovoltaic,7775.472,9386.319,14002.829,16183.384


### Asia

In [25]:
country_data(en_df, "north america")

,technology,2018,2019,2020,2021
3762,total renewable energy,1234086.732,1254649.757,1333705.195,1382303.006
3763,hydropower,735874.367,716430.804,722077.426,692112.951
3764,renewable hydropower,714260.412,695547.408,700892.982,671384.381
3765,pumped storage,21613.955,20883.396,21184.444,20728.570
3766,marine,20.011,1.011,0.011,0.011
3767,wind,322391.167,347799.875,397280.762,438641.900
3768,onshore wind energy,322288.500,347697.208,397136.047,438497.185
3769,offshore wind energy,102.667,102.667,144.715,144.715
3770,solar,92531.305,111915.574,140014.020,177621.519
3771,solar photovoltaic,88563.028,108323.116,136527.453,174391.697


### Ethiopia

In [26]:
country_data(en_df, "ethiopia")

,technology,2018,2019,2020,2021
399,total renewable energy,13259.114,14552.808,15071.945,15817.317
400,hydropower,12680.954,13655.000,14403.700,14942.830
401,renewable hydropower,12680.954,13655.000,14403.700,14942.830
402,pumped storage,0.000,0.000,0.000,0.000
403,marine,0.000,0.000,0.000,0.000
404,wind,532.524,848.071,609.041,817.201
405,onshore wind energy,532.524,848.071,609.041,817.201
406,offshore wind energy,0.000,0.000,0.000,0.000
407,solar,19.754,19.754,32.295,33.895
408,solar photovoltaic,19.754,19.754,32.295,33.895


### Brazil

In [27]:
country_data(en_df, "brazil")

,technology,2018,2019,2020,2021
4484,total renewable energy,495314.162,515479.450,522981.626,507666.628
4485,hydropower,388971.076,397877.057,396382.000,362818.451
4486,renewable hydropower,388971.076,397877.057,396382.000,362818.451
4487,pumped storage,0.000,0.000,0.000,0.000
4488,marine,0.044,0.044,0.044,0.044
4489,wind,48475.141,55985.622,57050.000,72285.975
4490,onshore wind energy,48475.141,55985.622,57050.000,72285.975
4491,offshore wind energy,0.000,0.000,0.000,0.000
4492,solar,3471.739,6664.991,10758.396,16760.829
4493,solar photovoltaic,3471.739,6664.991,10758.396,16760.829


### Canada

In [28]:
country_data(en_df, "canada")

,technology,2018,2019,2020,2021
3800,total renewable energy,434446.0,429802.999,437079.001,433680.0
3801,hydropower,385967.0,381843.000,386617.000,382867.0
3802,renewable hydropower,385856.0,381732.000,386506.000,382756.0
3803,pumped storage,111.0,111.000,111.000,111.0
3804,marine,20.0,1.000,0.000,0.0
3805,wind,33468.0,32873.000,35761.000,34753.0
3806,onshore wind energy,33468.0,32873.000,35761.000,34753.0
3807,offshore wind energy,0.0,0.000,0.000,0.0
3808,solar,4107.0,4415.000,4790.000,6043.0
3809,solar photovoltaic,4107.0,4415.000,4790.000,6043.0


In [29]:
country_perc(en_df, "canada")

,technology,2018,2019,2020,2021
3800,total renewable energy,100.00,100.00,100.00,100.00
3801,hydropower,88.84,88.84,88.45,88.28
3802,renewable hydropower,88.82,88.82,88.43,88.26
3803,pumped storage,0.03,0.03,0.03,0.03
3804,marine,0.00,0.00,0.00,0.00
3805,wind,7.70,7.65,8.18,8.01
3806,onshore wind energy,7.70,7.65,8.18,8.01
3807,offshore wind energy,0.00,0.00,0.00,0.00
3808,solar,0.95,1.03,1.10,1.39
3809,solar photovoltaic,0.95,1.03,1.10,1.39


### Ethiopia

In [30]:
country_data(en_df, "ethiopia")

,technology,2018,2019,2020,2021
399,total renewable energy,13259.114,14552.808,15071.945,15817.317
400,hydropower,12680.954,13655.000,14403.700,14942.830
401,renewable hydropower,12680.954,13655.000,14403.700,14942.830
402,pumped storage,0.000,0.000,0.000,0.000
403,marine,0.000,0.000,0.000,0.000
404,wind,532.524,848.071,609.041,817.201
405,onshore wind energy,532.524,848.071,609.041,817.201
406,offshore wind energy,0.000,0.000,0.000,0.000
407,solar,19.754,19.754,32.295,33.895
408,solar photovoltaic,19.754,19.754,32.295,33.895


In [31]:
country_perc(en_df, "ethiopia")

,technology,2018,2019,2020,2021
399,total renewable energy,100.00,100.00,100.00,100.00
400,hydropower,95.64,93.83,95.57,94.47
401,renewable hydropower,95.64,93.83,95.57,94.47
402,pumped storage,0.00,0.00,0.00,0.00
403,marine,0.00,0.00,0.00,0.00
404,wind,4.02,5.83,4.04,5.17
405,onshore wind energy,4.02,5.83,4.04,5.17
406,offshore wind energy,0.00,0.00,0.00,0.00
407,solar,0.15,0.14,0.21,0.21
408,solar photovoltaic,0.15,0.14,0.21,0.21


In [41]:
list(set(list(gen_df['country'].unique()))-set(['world', 
                                          'africa', 
                                          'asia', 
                                          'middle east', 
                                          'europe', 
                                          'european union',
                                          'north america',
                                          'south america', 
                                          'central america and the caribbean', 
                                          'oceania']))

['germany',
 'china',
 'maldives',
 'suriname',
 'azerbaijan',
 'tuvalu',
 'zambia',
 'luxembourg',
 'south africa',
 'serbia',
 'norway',
 'tokelau',
 'paraguay',
 'kenya',
 'comoros',
 'eritrea',
 'seychelles',
 'mexico',
 'samoa',
 'uruguay',
 'uganda',
 'bangladesh',
 "lao people's democratic republic",
 'netherlands',
 'puerto rico',
 'saint barthelemy',
 'french guiana',
 'cuba',
 'philippines',
 'switzerland',
 'sao tome and principe',
 'mongolia',
 'belize',
 'sierra leone',
 'north macedonia',
 'lithuania',
 'chinese taipei',
 'djibouti',
 'kyrgyzstan',
 'turks and caicos islands',
 'burkina faso',
 'indonesia',
 'panama',
 'denmark',
 'turkey',
 'kuwait',
 'dominican republic',
 'eurasia',
 'ethiopia',
 'italy',
 'sweden',
 'anguilla',
 'democratic republic of the congo',
 'afghanistan',
 'reunion',
 'namibia',
 'bulgaria',
 'honduras',
 'united arab emirates',
 'vanuatu',
 'china, hong kong sar',
 'republic of moldova',
 'uzbekistan',
 'ireland',
 'somalia',
 'jordan',
 'mya